In [18]:
s = 'ABCD'
s2 = '-asdfABCD'
s3 = '#'
s4 = 'abcd-'
s5 = 'ABD-'

In [19]:
import re

In [20]:
print(re.match('[A-Z]*[a-z]+|-[a-z][A-Z]*-*', s2))

<re.Match object; span=(0, 2), match='-a'>


In [21]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[A-Z]*[a-z]*|-[a-z][A-Z]*-*', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(0, 4), match='abcd'>
<re.Match object; span=(0, 3), match='ABD'>


In [22]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('^[A-Z]+|^-|^[a-z]', elem))

<re.Match object; span=(0, 4), match='ABCD'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 3), match='ABD'>


In [23]:
for elem in [s, s2, s3, s4, s5]:
    print(re.match('[^#]', elem))

<re.Match object; span=(0, 1), match='A'>
<re.Match object; span=(0, 1), match='-'>
None
<re.Match object; span=(0, 1), match='a'>
<re.Match object; span=(0, 1), match='A'>


In [24]:
# comment for recommit

In [25]:
import numpy as np

In [63]:
# Start reading sequences
with open('test1.txt', 'r') as f:
    a = f.readlines()
    


In [69]:
[elem.strip() for elem in a]

['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT']

In [70]:
a

['AAA\n',
 'ACC\n',
 'ACG\n',
 'ACT\n',
 '#\n',
 'AAA-\n',
 'A-CC\n',
 'ACG-\n',
 'A-CT\n']

In [77]:
a = list(map(str.strip, a))
a

['AAA', 'ACC', 'ACG', 'ACT', '#', 'AAA-', 'A-CC', 'ACG-', 'A-CT']

array([['AAA', 'ACC', 'ACG', 'ACT']], dtype='<U3')

In [100]:
c[0][0]

'AAA'

In [30]:
for i in a:
    print(i if re.match('[^#]', i) else None)

AAA

ACC

ACG

ACT

None
AAA-

A-CC

ACG-

A-CT



In [55]:
def array_regex(i):
    return np.array([True if re.match('[^#]', i) else False])

ar = np.vectorize(array_regex)

b = ar(a)

In [56]:
print(a)
print(b)

['AAA\n' 'ACC\n' 'ACG\n' 'ACT\n' '#\n' 'AAA-\n' 'A-CC\n' 'ACG-\n' 'A-CT\n']
[ True  True  True  True False  True  True  True  True]


array(['AAA\n', 'ACC\n', 'ACG\n', 'ACT\n', 'AAA-\n', 'A-CC\n', 'ACG-\n',
       'A-CT\n'], dtype=object)

In [45]:
b = np.array([])
for i in range(len(a)):
    print(i)

0
1
2
3
4
5
6
7
8


TypeError: slice indices must be integers or None or have an __index__ method